In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from defaults import motion_list, regressor_list
from heatmaps import generate_heatmap, reshape_corrs
from subjects import generate_subject_list_for_range

Fill in these variables before running the cells below:

In [ ]:
correlation_matrix = "/data3/cnl/fmriprep/benchmark_scripts/corrs.mat"
save_path = False # "{some/path}.png" or False

In [ ]:
generate_heatmap(
    reshape_corrs(correlation_matrix),
    regressor_list + motion_list,
    generate_subject_list_for_range(
        (25427, 25456), (1, 2)
    ),
    save_path
)